<a href="https://colab.research.google.com/github/sanjayashokshetty/A-new-Language/blob/master/Copy_of_json_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
# from scipy import spatial
# import torch
# import spacy
# en_nlp = spacy.load('en')

## Convert Json to Pandas Dataframe

In [60]:

# # Load the Drive helper and mount
from google.colab import drive

# # This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls "/content/drive/My Drive/NLP/data"

train.csv  valid.csv


In [0]:
#@title
!ls drive/My Drive

ls: cannot access 'drive/My//': No such file or directory
ls: cannot access 'Drive': No such file or directory


In [0]:
train = pd.read_json("data/train-v1.1.json")

In [0]:
valid = pd.read_json("data/dev-v1.1.json")

In [0]:
train.shape, valid.shape

((48, 2), (48, 2))

In [0]:
train.head(3)

,data,version
0,"{'title': 'Super_Bowl_50', 'paragraphs': [{'co...",1.1
1,"{'title': 'Warsaw', 'paragraphs': [{'context':...",1.1
2,"{'title': 'Normans', 'paragraphs': [{'context'...",1.1


In [0]:
train.iloc[1,0]['paragraphs'][0]

In [0]:
# valid.iloc[1,0]['paragraphs'][0]

In [0]:
contexts = []
questions = []
answers_text = []
answers_start = []
for i in range(train.shape[0]):
    topic = train.iloc[i,0]['paragraphs']
    for sub_para in topic:
        for q_a in sub_para['qas']:
            questions.append(q_a['question'])
            answers_start.append(q_a['answers'][0]['answer_start'])
            answers_text.append(q_a['answers'][0]['text'])
            contexts.append(sub_para['context'])   
df = pd.DataFrame({"context":contexts, "question": questions, "answer_start": answers_start, "text": answers_text})

In [0]:
df.shape

(10570, 4)

In [0]:
df.to_csv("data/train.csv", index = None)

## Train  a word2vec model


In [0]:
train_df = pd.read_csv("/content/drive/My Drive/NLP/data/trainp.csv")
valid_df = pd.read_csv("/content/drive/My Drive/NLP/data/testp.csv")

In [0]:
paras = list(train_df["pcontext"].drop_duplicates().reset_index(drop= True))

In [0]:
paras=paras+list(valid_df["pcontext"].drop_duplicates().reset_index(drop= True))

In [0]:
len(paras)

119124

In [0]:
paras_raw = " ".join(paras)

In [0]:
len(paras_raw)

15244989

In [0]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
sentences=paras_raw.split(".")

In [0]:
from nltk.tokenize import word_tokenize 
filtered_sentence=[[word_tokenize(sentences[i])] for i in range(len(sentences))]

In [0]:
filtered_sentence[1]

[['Atop',
  'Main',
  'Buildings',
  'gold',
  'dome',
  'golden',
  'statue',
  'Virgin',
  'Mary']]

In [0]:
from gensim.models import Word2Vec
model = Word2Vec(paras, min_count=1,seed=123)
print(model)


Word2Vec(vocab=1412, size=100, alpha=0.025)


In [0]:
model.save('drive/My Drive/NLP/wordvec/word2vec.bin')

In [59]:
model

## Redundant to be removed later


In [0]:
from nltk.corpus import stopwords
from __future__ import print_function
def create_corpus(corpus):
    corpus_out=[]
    sw=stopwords.words('english')
    count=1
    new_reviews=[]
    for review in corpus:
        review_sents=review.split(".")
        new_review_sent=[]
        for sent in review_sents:
            temp=sent.split(" ")
            for word in temp:
                if word=='' or word==' ' or word in sw:
                    temp.remove(word)
            new_review_sent+=temp
            corpus_out.append(temp)
        new_reviews.append(new_review_sent)
        s=str(count)+'/'+str(len(corpus))+' done!'
        if count%1==0:
          print(s,end='\r')
        count+=1
    return corpus_out,new_reviews


In [0]:
infersent.build_vocab(sentences, tokenize=True)

Found 88993(/109718) words with glove vectors
Vocab size : 88993


In [0]:
dict_embeddings = {}
for i in range(len(sentences)):
    print(i)
    dict_embeddings[sentences[i]] = infersent.encode([sentences[i]], tokenize=True)

In [0]:
questions = list(df["question"])

In [0]:
len(questions)

87599

In [0]:
for i in range(len(questions)):
    print(i)
    dict_embeddings[questions[i]] = infersent.encode([questions[i]], tokenize=True)

In [0]:
dict_embeddings['Architecturally, the school has a Catholic character.'][0]

array([ 0.05519997,  0.05013141,  0.04787038, ...,  0.00821208,
       -0.03642813,  0.04468501], dtype=float32)

In [0]:
d1 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings) if i % 2 == 0}
d2 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings) if i % 2 == 1}

In [0]:
d1

In [0]:
d2

In [0]:
with open('data/dict_embeddings1.pickle', 'wb') as handle:
    pickle.dump(d1, handle)

In [0]:
with open('data/dict_embeddings2.pickle', 'wb') as handle:
    pickle.dump(d2, handle)

In [0]:
del dict_embeddings

In [0]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
